### Import Required Libraries and Set Up Environment Variables

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pip install python-dotenv

In [5]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
import numpy as np
import ast
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [6]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
CME_url = base_url + CME + "?startDate=" + startDate + "&endDate=" + endDate + "&api_key=" + NASA_API_KEY

In [7]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(CME_url)

In [8]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [9]:
# Preview ONLY the first element from the cme_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=5))

{
     "activityID": "2013-05-01T03:12:00-CME-001",
     "catalog": "M2M_CATALOG",
     "startTime": "2013-05-01T03:12Z",
     "instruments": [
          {
               "displayName": "SOHO: LASCO/C2"
          },
          {
               "displayName": "SOHO: LASCO/C3"
          },
          {
               "displayName": "STEREO A: SECCHI/COR2"
          },
          {
               "displayName": "STEREO B: SECCHI/COR2"
          }
     ],
     "sourceLocation": "",
     "activeRegionNum": null,
     "note": "",
     "submissionTime": "2013-08-07T16:54Z",
     "versionId": 1,
     "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
     "cmeAnalyses": [
          {
               "isMostAccurate": true,
               "time21_5": "2013-05-01T07:07Z",
               "latitude": 12.0,
               "longitude": -120.0,
               "halfAngle": 36.0,
               "speed": 860.0,
               "type": "C",
               "featureCode": "null",
            

In [10]:
# Convert cme_json to a Pandas DataFrame
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [11]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [12]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents'
# and adds the elements individually to a list of dictionaries where each row is one element

# Initialize an empty list to store the expanded rows
rows_expanded = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    # Get the 'linkedEvents' value for the current index
    linked_events = cme_df.loc[i, 'linkedEvents']
    startTime = cme_df.loc[i, 'startTime']
    activityID = cme_df.loc[i, 'activityID']

    # Iterate over each dictionary in the list
    for item in linked_events:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      rows_expanded.append({'linkedEvents': item, 'startTime': startTime, 'activityID': activityID})
# Create a new DataFrame from the expanded rows
cme_df = pd.DataFrame(rows_expanded)
cme_df.head()

,linkedEvents,startTime,activityID
0,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-01T03:12Z,2013-05-01T03:12:00-CME-001
1,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-03T22:36Z,2013-05-03T22:36:00-CME-001
2,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-09T19:29Z,2013-05-09T19:29:00-CME-001
3,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T02:54Z,2013-05-13T02:54:00-CME-001
4,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T02:54Z,2013-05-13T02:54:00-CME-001


In [13]:
def extract_activityID_from_dict(input_obj):
    processed_dict = None

    # Check if the input is a string and try to parse it
    if isinstance(input_obj, str):
        try:
            # Using ast.literal_eval is safer than eval() for converting string
            # representations of Python literals, including dictionaries.
            processed_dict = ast.literal_eval(input_obj)
        except (ValueError, SyntaxError):
             # If literal_eval fails, it might be a JSON formatted string.
            try:
                processed_dict = json.loads(input_obj)
            except json.JSONDecodeError:
                print(f"Error: Input was a string that could not be parsed. Received: {input_obj}")
                return None
    # If it's already a dictionary, just use it.
    elif isinstance(input_obj, dict):
        processed_dict = input_obj

    # If we have a dictionary now, try to get the value.
    if isinstance(processed_dict, dict):
        return processed_dict.get('activityID')
    else:
        # This will catch cases where the input was not a dict or a parsable string.
        print(f"Error: Input was not a dictionary or a parsable string. Received: {type(input_obj)} with value: {input_obj}")
        return None


In [14]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_obj):
    try:
        activityID = input_obj.get('activityID', None)
        return activityID
    except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error processing input dictionary: {input_obj}. Error: {e}")
        return None

extract_activityID_from_dict(cme_df.loc[0,'linkedEvents'])

'2013-05-04T04:52:00-IPS-001'

In [15]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_df.loc[:, 'GST_ActivityID'] = cme_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
cme_df.head()

,linkedEvents,startTime,activityID,GST_ActivityID
0,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-01T03:12Z,2013-05-01T03:12:00-CME-001,2013-05-04T04:52:00-IPS-001
1,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-03T22:36Z,2013-05-03T22:36:00-CME-001,2013-05-07T04:37:00-IPS-001
2,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-09T19:29Z,2013-05-09T19:29:00-CME-001,2013-05-12T23:30:00-IPS-001
3,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T02:54Z,2013-05-13T02:54:00-CME-001,2013-05-13T01:53:00-FLR-001
4,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T02:54Z,2013-05-13T02:54:00-CME-001,2013-05-13T04:12:00-SEP-001


In [16]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_df = cme_df.dropna(subset='GST_ActivityID')

In [17]:
# print out the datatype of each column in this DataFrame:
cme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1722 entries, 0 to 1721
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   linkedEvents    1722 non-null   object
 1   startTime       1722 non-null   object
 2   activityID      1722 non-null   object
 3   GST_ActivityID  1722 non-null   object
dtypes: object(4)
memory usage: 53.9+ KB


In [18]:
# Rename startTime to startTime_CME and activityID to cmeID
cme_df.rename(columns={'startTime':'startTime_CME'}, inplace=True)
cme_df.rename(columns={'activityID':'cmeID'}, inplace=True)
# Convert the 'GST_ActivityID' column to string format
cme_df['GST_ActivityID'] = pd.Series(cme_df['GST_ActivityID'], dtype="string")
# Convert startTime to datetime format
cme_df['startTime_CME'] = pd.to_datetime(cme_df['startTime_CME'])

# Drop linkedEvents

# Verify that all steps were executed correctly
cme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1722 entries, 0 to 1721
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   linkedEvents    1722 non-null   object             
 1   startTime_CME   1722 non-null   datetime64[ns, UTC]
 2   cmeID           1722 non-null   object             
 3   GST_ActivityID  1722 non-null   string             
dtypes: datetime64[ns, UTC](1), object(2), string(1)
memory usage: 53.9+ KB


In [19]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.
cme_df = cme_df[cme_df['GST_ActivityID'].str.contains('GST')]
cme_df.head()

,linkedEvents,startTime_CME,cmeID,GST_ActivityID
21,{'activityID': '2013-06-07T03:00:00-GST-001'},2013-06-02 20:24:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-07T03:00:00-GST-001
48,{'activityID': '2013-10-02T03:00:00-GST-001'},2013-09-29 22:40:00+00:00,2013-09-29T22:40:00-CME-001,2013-10-02T03:00:00-GST-001
90,{'activityID': '2013-12-08T00:00:00-GST-001'},2013-12-04 23:12:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-08T00:00:00-GST-001
148,{'activityID': '2014-02-19T03:00:00-GST-001'},2014-02-16 14:15:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-19T03:00:00-GST-001
151,{'activityID': '2014-02-20T03:00:00-GST-001'},2014-02-18 01:25:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-20T03:00:00-GST-001


### GST Data

In [20]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
GST_url = base_url + GST + "?startDate=" + startDate + "&endDate=" + endDate + "&api_key=" + NASA_API_KEY

In [21]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
get_GST_response = requests.get(GST_url)

In [22]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = get_GST_response.json()

In [23]:
# Preview ONLY the first element from the gst_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=5))

{
     "gstID": "2013-06-01T01:00:00-GST-001",
     "startTime": "2013-06-01T01:00Z",
     "allKpIndex": [
          {
               "observedTime": "2013-06-01T01:00Z",
               "kpIndex": 6.0,
               "source": "NOAA"
          }
     ],
     "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
     "linkedEvents": [
          {
               "activityID": "2013-05-31T15:45:00-HSS-001"
          }
     ],
     "submissionTime": "2013-07-15T19:26Z",
     "versionId": 1
}


In [24]:
# Convert gst_json to a Pandas DataFrame
gst_df = pd.DataFrame(gst_json)
# Keep only the columns: gstID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [25]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [26]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True).dropna()
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [27]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df.loc[:, 'CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
gst_df.head()
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df.dropna(subset='CME_ActivityID')
gst_df.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [30]:
# Convert the 'CME_ActivityID' column to string format
gst_df['CME_ActivityID'] = pd.Series(gst_df['CME_ActivityID'], dtype="string")
# Convert the 'gstID' column to string format
gst_df['gstID'] = pd.Series(gst_df['gstID'], dtype="string")
# Convert startTime to datetime format
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])
# Rename startTime to startTime_GST
gst_df.rename(columns={'startTime':'startTime_GST'}, inplace=True)
# Drop linkedEvents

# Verify that all steps were executed correctly
gst_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    string             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   linkedEvents    205 non-null    object             
 3   CME_ActivityID  205 non-null    string             
dtypes: datetime64[ns, UTC](1), object(1), string(2)
memory usage: 6.5+ KB


In [31]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME')]
gst_df.head()

,gstID,startTime_GST,linkedEvents,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,{'activityID': '2013-12-04T23:12:00-CME-001'},2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,{'activityID': '2014-02-16T14:15:00-CME-001'},2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,{'activityID': '2014-02-18T01:25:00-CME-001'},2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [32]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
final_df = pd.merge(cme_df, gst_df, left_on=['GST_ActivityID', 'cmeID'], right_on=['gstID', 'CME_ActivityID'])
final_df.head()

,linkedEvents_x,startTime_CME,cmeID,GST_ActivityID,gstID,startTime_GST,linkedEvents_y,CME_ActivityID
0,{'activityID': '2013-06-07T03:00:00-GST-001'},2013-06-02 20:24:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-07T03:00:00-GST-001,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
1,{'activityID': '2013-10-02T03:00:00-GST-001'},2013-09-29 22:40:00+00:00,2013-09-29T22:40:00-CME-001,2013-10-02T03:00:00-GST-001,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
2,{'activityID': '2013-12-08T00:00:00-GST-001'},2013-12-04 23:12:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-08T00:00:00-GST-001,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,{'activityID': '2013-12-04T23:12:00-CME-001'},2013-12-04T23:12:00-CME-001
3,{'activityID': '2014-02-19T03:00:00-GST-001'},2014-02-16 14:15:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-19T03:00:00-GST-001,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,{'activityID': '2014-02-16T14:15:00-CME-001'},2014-02-16T14:15:00-CME-001
4,{'activityID': '2014-02-20T03:00:00-GST-001'},2014-02-18 01:25:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-20T03:00:00-GST-001,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,{'activityID': '2014-02-18T01:25:00-CME-001'},2014-02-18T01:25:00-CME-001


In [33]:
# Verify that the new DataFrame has the same number of rows as cme and gst
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   linkedEvents_x  61 non-null     object             
 1   startTime_CME   61 non-null     datetime64[ns, UTC]
 2   cmeID           61 non-null     object             
 3   GST_ActivityID  61 non-null     string             
 4   gstID           61 non-null     string             
 5   startTime_GST   61 non-null     datetime64[ns, UTC]
 6   linkedEvents_y  61 non-null     object             
 7   CME_ActivityID  61 non-null     string             
dtypes: datetime64[ns, UTC](2), object(3), string(3)
memory usage: 3.9+ KB


### Computing the time it takes for a CME to cause a GST

In [34]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
final_df['timeDiff']  = final_df['startTime_GST'] - final_df['startTime_CME']
final_df.head()

,linkedEvents_x,startTime_CME,cmeID,GST_ActivityID,gstID,startTime_GST,linkedEvents_y,CME_ActivityID,timeDiff
0,{'activityID': '2013-06-07T03:00:00-GST-001'},2013-06-02 20:24:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-07T03:00:00-GST-001,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001,4 days 06:36:00
1,{'activityID': '2013-10-02T03:00:00-GST-001'},2013-09-29 22:40:00+00:00,2013-09-29T22:40:00-CME-001,2013-10-02T03:00:00-GST-001,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001,2 days 04:20:00
2,{'activityID': '2013-12-08T00:00:00-GST-001'},2013-12-04 23:12:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-08T00:00:00-GST-001,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,{'activityID': '2013-12-04T23:12:00-CME-001'},2013-12-04T23:12:00-CME-001,3 days 00:48:00
3,{'activityID': '2014-02-19T03:00:00-GST-001'},2014-02-16 14:15:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-19T03:00:00-GST-001,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,{'activityID': '2014-02-16T14:15:00-CME-001'},2014-02-16T14:15:00-CME-001,2 days 12:45:00
4,{'activityID': '2014-02-20T03:00:00-GST-001'},2014-02-18 01:25:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-20T03:00:00-GST-001,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,{'activityID': '2014-02-18T01:25:00-CME-001'},2014-02-18T01:25:00-CME-001,2 days 01:35:00


In [35]:
# Use describe() to compute the mean and median time
# that it takes for a CME to cause a GST.
final_df[['timeDiff']].describe()

,timeDiff
count,61
mean,2 days 23:29:26.557377049
std,0 days 23:53:09.336914240
min,1 days 08:36:00
25%,2 days 04:00:00
50%,2 days 19:51:00
75%,3 days 13:35:00
max,6 days 03:00:00


### Exporting data in csv format

In [36]:
# Export data to CSV without the index
final_df.to_csv('final_df.csv', index=False)